**Описание задачи**

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

**Входные данные**

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

**Важно:**

    * Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    
    * Если товар не встречался в обучающей выборке, его популярность равна 0.
    
    * Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    
    * Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и
    k в recall@k / precision@k.

**Условие своими словами**

Есть обучающая выборка, по ней мы строим два словаря частот: частоты появлений в просмотренных, частоты появления в покупках. Это в каком-то смысле обучение рекомендательной системы (РС).

Далее нужно сделать применение РС к выборкам (обучающей и тестовой). РС работает так: просмотренные объекты в сессии сортируются по убыванию частоты из словаря РС. Это и есть рекомендация.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [94]:
train=pd.read_csv('coursera_sessions_train.txt',sep=';',header=None)

In [95]:
train

,0,1
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
...,...,...
49995,"32291,60520,32291,38220",32291
49996,"60479,28288,102804,102805",NaN
49997,"980,20025,980,20025,980,20025,980,20025",NaN
49998,"8844,42500,8838,8172,29237,352,8847,6681,8827,...",NaN


In [96]:
train.columns=['viewed','bought']

In [97]:
train.head()

,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [98]:
test=pd.read_csv('coursera_sessions_test.txt',sep=';',header=None,names=['viewed','bought'])

In [99]:
test.head()

,viewed,bought
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN


In [100]:
test=test.dropna() # Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.

**Задание**

**На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)**

In [101]:
import collections 


In [102]:
train_viewed=pd.Series(collections.Counter(train.viewed.str.cat(sep = ",").split(',')))
train_bought=pd.Series(collections.Counter(train.bought.str.cat(sep = ",").split(',')))

# загнал в Сериес, т.к. пандас норм принимает массив, анхэширует его и находит значение по индексу, словарь не смог

**Задание 2**

Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

Просмотрры все посчтитали, теперь можем удалять строки где есть nan

In [103]:
train1=train.dropna()

In [104]:
train1['viewed_list'] = train1.apply(lambda x: list(train_viewed[pd.unique(x.viewed.split(','))].
                                                    sort_values(ascending=False, kind='mergesort').index), axis=1)

In [105]:
train1.head()

,viewed,bought,viewed_list
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]"
19,"138,198,199,127",199,"[127, 138, 198, 199]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ..."
33,"352,353,352",352,"[352, 353]"


In [132]:
train1['bought_list']=train1.apply(lambda x: list(train_bought[pd.unique(x.viewed.split(','))].
                           sort_values(ascending=False, kind='mergesort').index), axis=1)


In [124]:
train1.head()

,viewed,bought,viewed_list,bought_list,Precision@5,Recall@5,Precision@1,Recall@1,Precision@5_b,Recall@5_b,Precision@1_b,Recall@1_b
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]","[67, 63, 60, 68, 66, 65, 64, 62, 61, 59]",0.4,0.666667,1,0.333333,0.6,1.0,1,0.333333
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]","[86, 93, 85, 92, 91, 90, 89, 88, 87, 84]",0.0,0.000000,0,0.000000,0.2,1.0,1,1.000000
19,"138,198,199,127",199,"[127, 138, 198, 199]","[127, 199, 138, 198]",0.2,1.000000,0,0.000000,0.2,1.0,0,0.000000
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...","[303, 312, 311, 310, 309, 308, 307, 306, 305, ...",0.2,1.000000,1,1.000000,0.2,1.0,1,1.000000
33,"352,353,352",352,"[352, 353]","[352, 353]",0.2,1.000000,1,1.000000,0.2,1.0,1,1.000000


**Задание 3.**

Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

**Пример метрик, подсчитанный для кортежа:**

Просмотры: ['59', '60', '61', '62', '60', '63', '64', '65', '66', '61', '67', '68', '67']

Покупки: ['67','60', '63']

Количество посчитанных значений по просмотрам по всем сессиям должно равняться: [('59', 1), ('60', 2), ('61', 2), ('62', 1), ('63', 6), ('64', 3), ('65', 2), ('66', 2), ('67', 2), ('68', 2)]

Отсортированный массив должен получиться: ['63', '64', '60', '61', '65', '66', '67', '68', '59', '62']

Для подсчета Precision@5 берем только первые 5 просмотров в отсортированном массиве, находим количество пересечений с покупками, и делим на 5. Должно получиться 0.4.

Для подсчета Recall@5 берем также первые 5 просмотров, находим количество пересечений с покупками и делим на количество покупок. Должно получиться 0.666.

Для подсчета Precision@1 берем первый просмотр и находим есть ли он в списке покупок. Делим на 1. Должно получиться 1.0

Для подсчета Recall@1 берем первый просмотр и находим есть ли он в списке покупок. Делим на количество покупок. Должно 
получиться 0.333.

**TRAIN**

**по популярности (частота появления в просмотренных)**

In [108]:
train1['Precision@5']=train1.apply(lambda x: len(set(x.viewed_list[:5]).intersection(x.bought.split(',')))/5, axis=1)
train1['Recall@5']=train1.apply(lambda x: len(set(x.viewed_list[:5]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)
train1['Precision@1']=train1.apply(lambda x: len(set(x.viewed_list[:1]).intersection(x.bought.split(','))), axis=1)
train1['Recall@1']=train1.apply(lambda x: len(set(x.viewed_list[:1]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)

In [109]:
train1.head()

,viewed,bought,viewed_list,bought_list,Precision@5,Recall@5,Precision@1,Recall@1
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]","[67, 63, 60, 68, 66, 65, 64, 62, 61, 59]",0.4,0.666667,1,0.333333
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]","[86, 93, 85, 92, 91, 90, 89, 88, 87, 84]",0.0,0.000000,0,0.000000
19,"138,198,199,127",199,"[127, 138, 198, 199]","[127, 199, 138, 198]",0.2,1.000000,0,0.000000
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...","[303, 312, 311, 310, 309, 308, 307, 306, 305, ...",0.2,1.000000,1,1.000000
33,"352,353,352",352,"[352, 353]","[352, 353]",0.2,1.000000,1,1.000000


**по покупаемости (частота появления в покупках)**

In [110]:
train1['Precision@5_b']=train1.apply(lambda x: len(set(x.bought_list[:5]).intersection(x.bought.split(',')))/5, axis=1)
train1['Recall@5_b']=train1.apply(lambda x: len(set(x.bought_list[:5]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)
train1['Precision@1_b']=train1.apply(lambda x: len(set(x.bought_list[:1]).intersection(x.bought.split(','))), axis=1)
train1['Recall@1_b']=train1.apply(lambda x: len(set(x.bought_list[:1]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)

In [111]:
train1.head()

,viewed,bought,viewed_list,bought_list,Precision@5,Recall@5,Precision@1,Recall@1,Precision@5_b,Recall@5_b,Precision@1_b,Recall@1_b
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]","[67, 63, 60, 68, 66, 65, 64, 62, 61, 59]",0.4,0.666667,1,0.333333,0.6,1.0,1,0.333333
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]","[86, 93, 85, 92, 91, 90, 89, 88, 87, 84]",0.0,0.000000,0,0.000000,0.2,1.0,1,1.000000
19,"138,198,199,127",199,"[127, 138, 198, 199]","[127, 199, 138, 198]",0.2,1.000000,0,0.000000,0.2,1.0,0,0.000000
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...","[303, 312, 311, 310, 309, 308, 307, 306, 305, ...",0.2,1.000000,1,1.000000,0.2,1.0,1,1.000000
33,"352,353,352",352,"[352, 353]","[352, 353]",0.2,1.000000,1,1.000000,0.2,1.0,1,1.000000


**TEST**

In [112]:
test.head()

,viewed,bought
7,"63,68,69,70,66,61,59,61,66,68","66,63"
14,"158,159,160,159,161,162",162
19,"200,201,202,203,204","201,205"
34,"371,372,371","371,373"
40,422,422


In [113]:
test['viewed_list'] = test.apply(lambda x: list(train_viewed[pd.unique(x.viewed.split(','))].
                                                    sort_values(ascending=False, kind='mergesort').index), axis=1)

In [114]:
test['bought_list'] = test.apply(lambda x: list(train_bought[pd.unique(x.viewed.split(','))].
                                                    sort_values(ascending=False, kind='mergesort').index), axis=1)

In [115]:
test.head()

,viewed,bought,viewed_list,bought_list
7,"63,68,69,70,66,61,59,61,66,68","66,63","[63, 61, 66, 68, 59, 69, 70]","[63, 68, 69, 70, 66, 61, 59]"
14,"158,159,160,159,161,162",162,"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]"
19,"200,201,202,203,204","201,205","[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]"
34,"371,372,371","371,373","[371, 372]","[371, 372]"
40,422,422,[422],[422]


**по популярности (частота появления в просмотренных)**

In [116]:
test['Precision@5']=test.apply(lambda x: len(set(x.viewed_list[:5]).intersection(x.bought.split(',')))/5, axis=1)
test['Recall@5']=test.apply(lambda x: len(set(x.viewed_list[:5]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)
test['Precision@1']=test.apply(lambda x: len(set(x.viewed_list[:1]).intersection(x.bought.split(','))), axis=1)
test['Recall@1']=test.apply(lambda x: len(set(x.viewed_list[:1]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)

In [117]:
test.head()

,viewed,bought,viewed_list,bought_list,Precision@5,Recall@5,Precision@1,Recall@1
7,"63,68,69,70,66,61,59,61,66,68","66,63","[63, 61, 66, 68, 59, 69, 70]","[63, 68, 69, 70, 66, 61, 59]",0.4,1.0,1,0.5
14,"158,159,160,159,161,162",162,"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]",0.2,1.0,0,0.0
19,"200,201,202,203,204","201,205","[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]",0.2,0.5,0,0.0
34,"371,372,371","371,373","[371, 372]","[371, 372]",0.2,0.5,1,0.5
40,422,422,[422],[422],0.2,1.0,1,1.0


**по покупаемости (частота появления в покупках)**

In [118]:
test['Precision@5_b']=test.apply(lambda x: len(set(x.bought_list[:5]).intersection(x.bought.split(',')))/5, axis=1)
test['Recall@5_b']=test.apply(lambda x: len(set(x.bought_list[:5]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)
test['Precision@1_b']=test.apply(lambda x: len(set(x.bought_list[:1]).intersection(x.bought.split(','))), axis=1)
test['Recall@1_b']=test.apply(lambda x: len(set(x.bought_list[:1]).intersection(x.bought.split(',')))/
                                len(x.bought.split(',')), axis=1)

**recommendations**

In [119]:
train1.describe()

,Precision@5,Recall@5,Precision@1,Recall@1,Precision@5_b,Recall@5_b,Precision@1_b,Recall@1_b
count,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000,3608.000000
mean,0.212084,0.823229,0.504989,0.436872,0.251996,0.924601,0.777716,0.663876
std,0.115494,0.339979,0.500044,0.469362,0.138355,0.217540,0.415838,0.424710
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,1.000000,0.000000,0.000000,0.200000,1.000000,1.000000,0.250000
50%,0.200000,1.000000,1.000000,0.154762,0.200000,1.000000,1.000000,1.000000
75%,0.200000,1.000000,1.000000,1.000000,0.200000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [120]:
test.describe()

,Precision@5,Recall@5,Precision@1,Recall@1,Precision@5_b,Recall@5_b,Precision@1_b,Recall@1_b
count,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000
mean,0.203820,0.798754,0.475307,0.412296,0.209823,0.819920,0.525239,0.457743
std,0.115059,0.361300,0.499458,0.466974,0.113651,0.342312,0.499431,0.471316
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,0.666667,0.000000,0.000000,0.200000,1.000000,0.000000,0.000000
50%,0.200000,1.000000,0.000000,0.000000,0.200000,1.000000,1.000000,0.333333
75%,0.200000,1.000000,1.000000,1.000000,0.200000,1.000000,1.000000,1.000000
max,0.800000,1.000000,1.000000,1.000000,0.800000,1.000000,1.000000,1.000000


In [136]:
def wrt(x,out):
    with open(out,'w') as f:
        ans=[]
        for i in range(3,-1,-1):
            ans.append(round(x.mean(),2)[i])
        out=' '.join(map(str,ans))
        f.write(out)

In [137]:
wrt(train1[train1.columns[4:8]],'answer1')
wrt(test[test.columns[4:8]],'answer2')
wrt(train1[train1.columns[8:]],'answer3')
wrt(test[test.columns[8:]],'answer4')

In [138]:
round(train1[train1.columns[4:8]].mean(),2)

Precision@5    0.21
Recall@5       0.82
Precision@1    0.50
Recall@1       0.44
dtype: float64

In [139]:
round(test[test.columns[4:8]].mean(),2)

Precision@5    0.20
Recall@5       0.80
Precision@1    0.48
Recall@1       0.41
dtype: float64

In [140]:
round(train1[train1.columns[8:]].mean(),2)

Precision@5_b    0.25
Recall@5_b       0.92
Precision@1_b    0.78
Recall@1_b       0.66
dtype: float64

In [141]:
round(test[test.columns[8:]].mean(),2)

Precision@5_b    0.21
Recall@5_b       0.82
Precision@1_b    0.53
Recall@1_b       0.46
dtype: float64